# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 2:</center>
### <center>  Propiedades en Venta</center>
### <center>  Grupo 18</center>
### <center>NLP</center>


#### **Integrantes:**

* Analía Acevedo
* Santiago Daniel Marczewski Neudeck
* Lucia Belén Napoli
* Brayan Ricaldi

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sklearn.preprocessing as skp
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from sklearn.model_selection import RandomizedSearchCV
from joblib import dump, load

pd.options.display.max_columns = None

Descargo los dataSet a usar

In [3]:
_x_train = pd.read_csv('./DataSet/TP2-NLP/x_train')
_x_train.drop(columns=['Unnamed: 0'],inplace=True)
_y_train = pd.read_csv('./DataSet/TP2-NLP/y_train')
_y_train.drop(columns=['Unnamed: 0'],inplace=True)
_x_test = pd.read_csv('./DataSet/TP2-NLP/x_test')
_x_test.drop(columns=['Unnamed: 0'],inplace=True)
_y_test = pd.read_csv('./DataSet/TP2-NLP/y_test')
_y_test.drop(columns=['Unnamed: 0'],inplace=True)
_y_train.rename(columns={'0':'pxm2'}, inplace=True)
_y_test.rename(columns={'0':'pxm2'}, inplace=True)

Creo una copia, asi no modifico el original

In [4]:
x_train = _x_train.copy()
y_train = _y_train.copy()
x_test = _x_test.copy()
y_test = _y_test.copy()

Descargo el dataSet de descripciones

In [5]:
df_descrip = pd.read_csv("./DataSet/TP2-NLP/descripciones.csv")
df_descrip.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


In [6]:
df_descrip.shape

(460154, 2)

A continuacion unire los datos de mi dataSet con su respectiva descripcion mediante su id.

In [7]:
x_train = pd.merge(x_train, df_descrip, on="id")

In [8]:
x_test = pd.merge(x_test, df_descrip, on="id")

Ahora si puedo eliminar el feature id

In [9]:
x_train.drop(columns='id', inplace=True)
x_test.drop(columns='id', inplace=True)
x_train.rename(columns={'property_description':'descripcion'}, inplace=True)
x_test.rename(columns={'property_description':'descripcion'}, inplace=True)

In [10]:
x_train.head()

,latitud,longitud,ambientes,dormitorios,superficie_cubierta,comuna,Departamento,PH,descripcion
0,-34.583010,-58.481533,4.0,3.0,70.0,15,1.0,0.0,Corredor Responsable: MAURO MARVISI - CSI 5574...
1,-34.660079,-58.485894,6.0,4.0,275.0,9,0.0,0.0,Corredor Responsable: Ariel Champanier / Andre...
2,-34.581856,-58.425088,3.0,2.0,97.0,14,1.0,0.0,Departamento de 3 ambientes con dependencia de...
3,-34.607547,-58.442194,2.0,1.0,42.0,6,1.0,0.0,Corredor Responsable: Mauro Marvisi CMCPSI 557...
4,-34.607416,-58.509680,2.0,1.0,38.0,11,1.0,0.0,Departamento de 2 ambientes al frente con balc...


# Sentimientos y emociones

Con la gran polemica que twitter esta generando estos dias decidimos hacer un estudio sobre si existe una relacion entre las emociones/sentimientos de las descripciones con sus respectivos precios.
Por eso tomamos un modelo ya preprocesado sobre un corpus de tweets y generaremos los siguientes features:

In [13]:
#Mas info: https://github.com/pysentimiento/pysentimiento
from pysentimiento import create_analyzer 

2022-12-08 19:08:59.004634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Sentimientos

In [11]:
# Crearemos una instancia del objeto a usar
# Explicacion de los siguientes parametros
# Task: Nombre de la tarea, en este caso pediremos los sentimientos de las descripciones
# Lang: Idioma que usaremos, es -> español
analyzer = create_analyzer(task="sentiment", lang="es")

Comenzamos con un ejemplo
Predecimos una frase

In [12]:
analyzer.predict("Qué gran jugador es Messi")

AnalyzerOutput(output=POS, probas={POS: 0.994, NEG: 0.003, NEU: 0.003})

Observamos que nos devuelve un objeto con sus atributos:
   - Output: Como categorizo la frase, en este caso considero que la frase es positiva
   - Probas: Un diccionario con las probabilidades de que la frase sea positiva, negativa y neutra.

Observamos otro ejemplo:

In [14]:
analyzer.predict("Esto es pésimo")

AnalyzerOutput(output=NEG, probas={NEG: 0.948, NEU: 0.048, POS: 0.004})

En este ejemplo considero la frase negativa y creemos que en definitiva si lo es.

Crearemos un feature con el porcentaje de sentimiento positivo de su descripcion.

Comenzamos con x_train.

In [19]:
sentimiento_positivo_x_train = []

In [ ]:
for descrip in list(x_train.descripcion):
    analizador = analyzer.predict(descrip)
    sentimiento_positivo_x_train.append(analizador.probas['POS'])

Vamos a guardar el resultado porque el proceso es muy costoso y para no guardar el dataSet completo guardaremos el feature sentimiento con su respectivo id para luego unirlo con el dataSet completo.

In [30]:
list_x_train_descrip = list(zip(list(_x_train.id), sentimiento_positivo_x_train))

In [31]:
x_train_sentiment = pd.DataFrame(list_x_train_descrip)

In [39]:
x_train_sentiment.rename(columns={0:'id', 1:'Sentimiento Positivo'}, inplace=True)

In [40]:
#Lo guardamos
#x_train_sentiment.to_csv("./DataSet/TP2-NLP/x_train_sentiment")

,id,Sentimiento Positivo
0,JhrnbHggMI/+a3HqPvSHpA==,0.296470
1,QgE3a9anrPSP3oqujv1C0g==,0.048648
2,GdDiEcbXr0qGtVgRjZYIXQ==,0.949890
3,EOc0KchotIHQCSq1oFQxyw==,0.244936
4,vLV4Bu3fyY5e9Stj/b/Ekw==,0.757725
...,...,...
58128,zDOIVJ3+aFbBvVOl+zNM4g==,0.991766
58129,KOM0QqwoP77Edwn85zAHag==,0.872351
58130,8SZBK9DZp4lmGvYzOpquOw==,0.959569
58131,avcVU8ThBABuYjg4y5T7vA==,0.942879


Haremos el mismo proceso con el dataSet x_test.

In [42]:
sentimiento_positivo_x_test = []

In [43]:
for descrip in list(x_test.descripcion):
    analizador = analyzer.predict(descrip)
    sentimiento_positivo_x_test.append(analizador.probas['POS'])

In [64]:
list_x_test_sentiment = list(zip(list(_x_test.id), sentimiento_positivo[-len(x_test):]))

In [65]:
x_test_sentiment = pd.DataFrame(x_test_sentiment)

In [66]:
x_test_sentiment.rename(columns={0:'id', 1:'Sentimiento Positivo'}, inplace=True)

In [67]:
x_test_sentiment.head()

,id,Sentimiento Positivo
0,L/i3QyPkam5OcxMoY4PD4A==,0.972614
1,qb5jsGial8hrGHZFt4kjXA==,0.016784
2,6sDtQxzc7YCbXMUl9V11ow==,0.093802
3,CqquLglwqJ/fzqhm2nMxog==,0.380551
4,nxWt1nd/+yFkzf2X6xdSfQ==,0.997235


In [61]:
#Lo guardamos
#x_test_sentiment.to_csv("./DataSet/TP2-NLP/x_test_sentiment")

## Emociones

In [14]:
# Crearemos una instancia del objeto a usar
# Explicacion de los siguientes parametros
# Task: Nombre de la tarea, en este caso pediremos las emociones de las descripciones
# Lang: Idioma que usaremos, es -> español
emotion_analyzer = create_analyzer(task="emotion", lang="es")

Un ejemplo:

In [105]:
emotion_analyzer.predict("Estoy muy triste")

AnalyzerOutput(output=sadness, probas={sadness: 0.953, fear: 0.015, anger: 0.010, others: 0.009, disgust: 0.007, joy: 0.003, surprise: 0.002})

Observamos que nos devuelve un objeto con 2 atributos:
   - Output: Como categorizo la frase, en este caso considero que la frase es triste
   - Probas: Un diccionario con las probabilidades de que la frase alegre, triste, sorpresa, miedo, enfado, otros, disgusto.

Crearemos 6 features, 1 por cada emocion y guardaremos en cada uno su respectiva probabilidad.

Comenzamos con x_train

In [83]:
emociones_x_train = []

In [84]:
for descrip in list(x_train.descripcion):
    analizador = emotion_analyzer.predict(descrip)
    probabilidades = analizador.probas
    emociones_x_train.append([probabilidades['joy'], probabilidades['sadness'], probabilidades['surprise'], probabilidades['fear'], probabilidades['anger'], probabilidades['others'], probabilidades['disgust']])

In [94]:
x_train_emotion = pd.DataFrame(emociones_x_train)
x_train_emotion.rename(columns={0:'alegria', 1:'tristeza', 2:'sorpresa', 3: 'miedo', 4:'enfado', 5: 'otros', 6:'disgusto'}, inplace=True)
x_train_emotion['id'] = _x_train.id

In [96]:
x_train_emotion

,alegria,tristeza,sorpresa,miedo,enfado,otros,disgusto,id
0,0.290528,0.005603,0.014617,0.002689,0.007023,0.676980,0.002560,JhrnbHggMI/+a3HqPvSHpA==
1,0.257622,0.005772,0.018778,0.002709,0.004777,0.707761,0.002582,QgE3a9anrPSP3oqujv1C0g==
2,0.532302,0.011435,0.034491,0.006770,0.006357,0.403338,0.005306,GdDiEcbXr0qGtVgRjZYIXQ==
3,0.296273,0.006642,0.033718,0.006992,0.004343,0.647592,0.004440,EOc0KchotIHQCSq1oFQxyw==
4,0.657242,0.011658,0.043448,0.009398,0.004551,0.267826,0.005878,vLV4Bu3fyY5e9Stj/b/Ekw==
...,...,...,...,...,...,...,...,...
58128,0.897437,0.004728,0.024074,0.003241,0.002847,0.065259,0.002414,zDOIVJ3+aFbBvVOl+zNM4g==
58129,0.647558,0.007072,0.024863,0.004166,0.003186,0.310427,0.002729,KOM0QqwoP77Edwn85zAHag==
58130,0.890901,0.003285,0.015248,0.002070,0.002225,0.084157,0.002115,8SZBK9DZp4lmGvYzOpquOw==
58131,0.429254,0.011146,0.032825,0.006015,0.006050,0.509681,0.005028,avcVU8ThBABuYjg4y5T7vA==


In [97]:
#Lo guardamos
#x_train_emotion.to_csv("./DataSet/TP2-NLP/x_train_emotion")

Ahora con x_test

In [15]:
emociones_x_test = []

In [16]:
for descrip in list(x_test.descripcion):
    analizador = emotion_analyzer.predict(descrip)
    probabilidades = analizador.probas
    emociones_x_test.append([probabilidades['joy'], probabilidades['sadness'], probabilidades['surprise'], probabilidades['fear'], probabilidades['anger'], probabilidades['others'], probabilidades['disgust']])

In [17]:
x_test_emotion = pd.DataFrame(emociones_x_test)
x_test_emotion.rename(columns={0:'alegria', 1:'tristeza', 2:'sorpresa', 3: 'miedo', 4:'enfado', 5: 'otros', 6:'disgusto'}, inplace=True)
x_test_emotion['id'] = _x_test.id

In [18]:
x_test_emotion.head()

,alegria,tristeza,sorpresa,miedo,enfado,otros,disgusto,id
0,0.879647,0.002303,0.011425,0.001547,0.001716,0.102002,0.001360,L/i3QyPkam5OcxMoY4PD4A==
1,0.052972,0.003644,0.010320,0.001605,0.003328,0.926906,0.001225,qb5jsGial8hrGHZFt4kjXA==
2,0.115030,0.007334,0.013896,0.002847,0.004355,0.854284,0.002254,6sDtQxzc7YCbXMUl9V11ow==
3,0.404646,0.006501,0.026653,0.004463,0.004921,0.548926,0.003889,CqquLglwqJ/fzqhm2nMxog==
4,0.968894,0.001639,0.016190,0.001075,0.001141,0.009884,0.001177,nxWt1nd/+yFkzf2X6xdSfQ==


In [21]:
#Lo guardamos
#x_test_emotion.to_csv("./DataSet/TP2-NLP/x_test_emotion")